In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.stats import unitary_group
import jax
import jax.numpy as jnp

from permanent import perm
from piquasso._math.permanent import permanent

def generate_random_unitary(n):
    return unitary_group.rvs(n, random_state=42)

def generate_multiplicity_vectors(n):
    rows = np.ones(n, dtype=np.uint64) * 3
    cols = np.ones(n, dtype=np.uint64) * 3
    return rows, cols

n_values = range(2, 15)  # Adjust upper limit as needed
times_gpu = []
times_cpu = []
times_piquasso = []

for n in n_values:
    matrix = generate_random_unitary(n)
    rows, cols = generate_multiplicity_vectors(n)
    
    # GPU (your implementation)
    try:
        jax.config.update('jax_platform_name', 'gpu')
        with jax.default_device(jax.devices('gpu')[0]):
            start = time.time()
            perm(matrix, rows, cols)
            times_gpu.append(time.time() - start)
    except Exception as e:
        times_gpu.append(np.nan)
    
    # CPU (your implementation)
    try:
        jax.config.update('jax_platform_name', 'cpu')
        with jax.default_device(jax.devices('cpu')[0]):
            start = time.time()
            perm(matrix, rows, cols)
            times_cpu.append(time.time() - start)
    except Exception as e:
        times_cpu.append(np.nan)
    
    # piquasso
    try:
        start = time.time()
        permanent(matrix, rows, cols)
        times_piquasso.append(time.time() - start)
    except Exception as e:
        times_piquasso.append(np.nan)

plt.figure(figsize=(10, 6))
plt.plot(n_values, times_gpu, label='Your GPU')
plt.plot(n_values, times_cpu, label='Your CPU')
plt.plot(n_values, times_piquasso, label='Piquasso CPU')
plt.yscale('log')
plt.xlabel('Matrix size n')
plt.ylabel('Time (s, log scale)')
plt.title('Permanent Benchmark')
plt.legend()
plt.grid(True)
plt.show()